## Import Pandas and NumPy

In [1]:
import pandas as pd
import numpy as np

## This is the data cleaning python workbook for the Open Data Week project: How to create maps in Python using Folium. This dataset was downloaded from the NYC Open Data page at: <br/><br/>https://data.cityofnewyork.us/Environment/Natural-Gas-Consumption-by-ZIP-Code-2010/uedp-fegm/data.<br/><br/> The cleaned dataset from this page can be found on my github at:<br/><br/>

In [2]:
energy = pd.read_csv(r'C:\Users\holli\OneDrive\Documents\DS with Python\Datasets\Natural_Gas_Consumption_by_ZIP_Code_-_2010.csv',
                     sep = ',')

In [3]:
energy.head()

,Zip Code,Building type (service class,Consumption (therms),Consumption (GJ),Utility/Data Source
0,10300,Commercial,470,50,National Grid
1,10335,Commercial,647,68,National Grid
2,10360,Large residential,"33,762","3,562",National Grid
3,11200,Commercial,"32,125","3,389",National Grid
4,11200,Institutional,"3,605",380,National Grid


In [4]:
energy.columns = ['Zip Code', 'Building type (service class)', 'Consumption (therms)', 'Consumption (GJ)',
                 'Utility/Data Source']

In [5]:
energy = energy.drop('Consumption (GJ)', axis = 1)

In [6]:
energy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1015 entries, 0 to 1014
Data columns (total 4 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Zip Code                       1015 non-null   object
 1   Building type (service class)  1015 non-null   object
 2   Consumption (therms)           1005 non-null   object
 3   Utility/Data Source            1015 non-null   object
dtypes: object(4)
memory usage: 31.8+ KB


## Turn Consumption (therms) column into a float

In [7]:
energy['Consumption (therms)'] = energy['Consumption (therms)'].str.replace(',', '')
energy['Consumption (therms)'].head()

0      470
1      647
2    33762
3    32125
4     3605
Name: Consumption (therms), dtype: object

In [8]:
energy['Consumption (therms)'] = energy['Consumption (therms)'].astype('float64')

## The first 50 rows do not have coordinates so they will not but useful to us with our maps

In [9]:
energy_coords = energy.loc[51:, :]

In [10]:
energy_coords.head()

,Zip Code,Building type (service class),Consumption (therms),Utility/Data Source
51,"11109(40.744414792409, -73.957702346686)",Commercial,45899.0,ConEd
52,"11429(40.709913120494, -73.738640316098)",Commercial,755.0,ConEd
53,"11226(40.646505002304, -73.957190099144)",Industrial,65835.0,National Grid
54,"10314(40.596490302985, -74.165991118795)",Institutional,2294516.0,National Grid
55,"11223(40.596939623165, -73.973311472047)",Commercial,2376036.0,National Grid


## Seperate the Zip Codes from the Coordinates, which are both in the Zip Code column

In [11]:
coords = energy_coords['Zip Code'].str.split('(', expand = True)

In [12]:
coords

,0,1
51,11109,"40.744414792409, -73.957702346686)"
52,11429,"40.709913120494, -73.738640316098)"
53,11226,"40.646505002304, -73.957190099144)"
54,10314,"40.596490302985, -74.165991118795)"
55,11223,"40.596939623165, -73.973311472047)"
...,...,...
1010,10029,"40.791610720558, -73.944032281065)"
1011,11365,"40.739871843711, -73.794621535156)"
1012,11434,"40.676945262114, -73.775306901692)"
1013,11221,"40.691288039511, -73.927617927925)"


## Make Zip Codes seperate column

In [13]:
energy_coords['Zip Code'] = coords[0]

C:\Users\holli\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Check the Building type column

In [14]:
energy_coords['Building type (service class)'].value_counts()

Commercial           338
Residential          198
Small residential     96
Large residential     87
Institutional         86
Industrial            81
Large Residential     78
Name: Building type (service class), dtype: int64

In [15]:
energy_coords['Building type (service class)'] = energy_coords['Building type (service class)'].str.replace('Large Residential', 'Large residential')

energy_coords['Building type (service class)'].value_counts()

C:\Users\holli\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Commercial           338
Residential          198
Large residential    165
Small residential     96
Institutional         86
Industrial            81
Name: Building type (service class), dtype: int64

## Seperate latitude and longitude and remove unneeded characters

In [16]:
coords1 = coords[1].str.split(',', expand = True)
coords1[1] = coords1[1].str.replace(')', '')

C:\Users\holli\anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will*not* be treated as literal strings when regex=True.
  


In [17]:
coords1

,0,1
51,40.744414792409,-73.957702346686
52,40.709913120494,-73.738640316098
53,40.646505002304,-73.957190099144
54,40.596490302985,-74.165991118795
55,40.596939623165,-73.973311472047
...,...,...
1010,40.791610720558,-73.944032281065
1011,40.739871843711,-73.794621535156
1012,40.676945262114,-73.775306901692
1013,40.691288039511,-73.927617927925


## Now we have our latitude and longitude in seperate columns and a part of our mapping dataset

In [18]:
energy_coords['Latitude'] = coords1[0]
energy_coords['Longitude'] = coords1[1]

C:\Users\holli\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\holli\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Reset our index

In [19]:
energy_coords = energy_coords.reset_index()
energy_coords = energy_coords.drop('index', axis = 1)
energy_coords.head()

,Zip Code,Building type (service class),Consumption (therms),Utility/Data Source,Latitude,Longitude
0,11109,Commercial,45899.0,ConEd,40.744414792409,-73.957702346686
1,11429,Commercial,755.0,ConEd,40.709913120494,-73.738640316098
2,11226,Industrial,65835.0,National Grid,40.646505002304,-73.957190099144
3,10314,Institutional,2294516.0,National Grid,40.596490302985,-74.165991118795
4,11223,Commercial,2376036.0,National Grid,40.596939623165,-73.973311472047


## We will replace the missing values in our Consumption (therms) column with median imputation

In [20]:
energy_coords.isnull().sum()

Zip Code                         0
Building type (service class)    0
Consumption (therms)             9
Utility/Data Source              0
Latitude                         0
Longitude                        0
dtype: int64

## Create a mask to get rows for missing therms values and fill the null values with the median

In [21]:
therms_null_mask = energy_coords['Consumption (therms)'].isnull()

In [22]:
energy_coords.loc[therms_null_mask, 'Consumption (therms)'] = energy_coords.loc[therms_null_mask, 
                                                            'Consumption (therms)'].fillna(
    np.median(energy_coords.loc[~therms_null_mask, 'Consumption (therms)']))

## Check our data types

In [23]:
energy_coords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Zip Code                       964 non-null    object 
 1   Building type (service class)  964 non-null    object 
 2   Consumption (therms)           964 non-null    float64
 3   Utility/Data Source            964 non-null    object 
 4   Latitude                       964 non-null    object 
 5   Longitude                      964 non-null    object 
dtypes: float64(1), object(5)
memory usage: 45.3+ KB


## Change Latitude and Longitude columns into floats

In [24]:
energy_coords['Latitude'] = energy_coords['Latitude'].astype('float64')
energy_coords['Longitude'] = energy_coords['Longitude'].astype('float64')

energy_coords.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 964 entries, 0 to 963
Data columns (total 6 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Zip Code                       964 non-null    object 
 1   Building type (service class)  964 non-null    object 
 2   Consumption (therms)           964 non-null    float64
 3   Utility/Data Source            964 non-null    object 
 4   Latitude                       964 non-null    float64
 5   Longitude                      964 non-null    float64
dtypes: float64(3), object(3)
memory usage: 45.3+ KB


## Now we are ready to export our dataset

In [27]:
energy_coords.to_csv('Natural_Gas_Consumption_by_ZIP_Code_-_2010_clean.csv', index = False)